In [ ]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append('/home/obesity')
from obesity.data_preproocessing import write_obesity_patient_snp_data
from obesity.snp_encoding_traintest_split import read_obesity_and_normal_snp_data, snp_encoding_and_labeling
from obesity.model_evaluation import predict_score_metric_array, create_result_table
from tensorflow.keras.models import load_model
from itertools import chain

In [ ]:
obesity_data_path, normal_data_path = write_obesity_patient_snp_data('/home/obesity/input_data/ped_after1021/TWB2_male_3060_bmi2430_exclude_combine1020_test_acgt_ped.ped','/home/obesity/input_data/MOlist.csv', '/home/obesity/snp_data/snp_data_after1021')

In [ ]:
O_data, N_data = read_obesity_and_normal_snp_data(obesity_data_path,normal_data_path)

In [ ]:
data_encoded, labels = snp_encoding_and_labeling(O_data,N_data)

In [ ]:
models_root = '/home/obesity/cv_results_10_fold/cv_results_10_fold_after1021/TWB2_male_3060_bmi2430_exclude_combine1020_acgt_ped/model'
predict_score_df = pd.DataFrame()
for m in range(1, 11):
    model_path = os.path.join(models_root, 'model_best_{}.h5'.format(m))
    model = load_model(model_path)
    predict_score = model.predict(data_encoded)
    predict_score_df['model_best_{}'.format(m)]= list(chain.from_iterable(predict_score))

In [ ]:
ensemble_models_test_result = pd.DataFrame()
predict_score_df['Mean']= predict_score_df.mean(axis=1)
metrics = predict_score_metric_array(np.array(predict_score_df['Mean']),labels)
ensemble_models_test_result['test_after1021'] = metrics

In [ ]:
ensemble_models_test_result.index =['accuracy','precision','sensitivity','specificity','f1','mcc']

In [ ]:
p = np.round(ensemble_models_test_result,2).T

In [ ]:
p.to_csv('/home/obesity/cv_results_10_fold/cv_results_10_fold_after1021/TWB2_male_3060_bmi2430_exclude_combine1020_acgt_ped/ensemble_models_test_result.csv')




In [ ]:
os.chdir('/home/obesity/cv_results_10_fold/cv_results_10_fold_after1021/TWB2_male_3060_bmi2430_exclude_combine1020_acgt_ped')
create_result_table()